In [1]:
import plots

# Interactive NFL Player Performance Plots
The following bivariate plots examine the relationship between pre-draft combine measurables and NFL career yardage (rushing yards + receiving yards for RBs and WRs, passing yards for QBs) for players drafted between 1999-2008. Career yardage totals are through the 2015-2016 season. Use the tabs to switch between positions, tools to pan and zoom, and mouse over points to view individual player data. All plots were created using [Bokeh](http://bokeh.pydata.org), a visualization library for Python.

The first plot looks at 40 yard dash time, which measures straight-line speed and is generally viewed as the most important combine test. For running backs, there appears to be a slight negative relationship between 40 time and yardage, indicating that faster running backs are more likely to have more productive careers. In contrast, there does not appear to be any clearly discernable relationship between yardage and 40 time for wide receivers or quarterbacks, suggesting that success for these positions may be more dependent on other characteristics such as skill, intelligence, or agility. In particular, it is interesting to note that Anquan Boldin, the slowest wide receiver in the dataset, had the highest career yardage total.

In [2]:
plots.plot_40dash()

Loading BokehJS ...

The twenty yard short shuttle test has players run 5 yards in one direction, 10 yards in the opposite direction, then 5 yards again in the original direction. It tests straight-line speed as well as quickness in terms of changing direction. There did not appear to be a strong relationship between 20 yard shuttle times and yardage for any position. Quarterbacks were again the slowest overall position group, though the disparity was much smaller than it was for the 40 yard dash.

In [ ]:
plots.plot_twentyss()

Vertical jump is a measure of a player's explosiveness. On its own, it did not demonstrate a strong relationship with career yardage for any position. As with the other physical tests, wide receivers tended to perform best and quarterbacks had a relatively high variance.

In [ ]:
plots.plot_vertical()

Plotting career yardage by height did not uncover any strong correlations, but did reveal significant differences in typical height by position. Running backs were the shortest position group, with most players between 68-73 inches tall. Only six running backs in the dataset were taller than 73 inches (6'1"). In contrast, wide receivers tended to be 70-75 inches tall, with 73 inches being by far the most common height. Quarterbacks, for whom added height provides a better view of the field, were the tallest position group with most being 74-77 inches.

In [4]:
plots.plot_height()

Loading BokehJS ...

Finally, career yardage totals were plotted against speed score, which is a metric used to measure speed that adjusts for players' weight. A 220 pound player who ran a 4.5s 40 yard dash would rate better by this metric than a 200 pound player who ran the exact same time. Speed score is calculated using the following formula:

$$\frac{(weight)*200}{(fortyyd)^4}$$

where *weight* is weight in pounds and *fortyyd* is 40 yard dash time in seconds.

As expected, speed score demonstrated a slightly stronger correlation with yardage for running backs than 40 yard dash time. In addition, the metric tended to rate running backs as well, or better, than wide receivers as opposed to the other combine tests that favored the comparitively lighter wide receivers. Quarterbacks fared quite poorly by this metric in comparision to the other position groups, with most having a speed score between 70-100.

In [3]:
plots.plot_speedscore()

Loading BokehJS ...